In [1]:
!pip install wfdb pyts tensorflow matplotlib scikit-learn pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [2]:
!pip install pandas==2.2.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 100.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.3.2
    Uninstalling pandas-2.3.2:
      Successfully uninstalled pandas-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wfdb 4.3.0 requires pandas>=2.2.3, but you have pandas 2.2.2 which is incompatible.


In [3]:
!pip install pandas==2.2.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 123.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [4]:
import wfdb, os
os.makedirs('data/ptbdb', exist_ok=True)
wfdb.dl_database('ptbdb', 'data/ptbdb')   # downloads PTB files into data/ptbdb/


Generating record list for: patient001/s0010_re
Generating record list for: patient001/s0014lre
Generating record list for: patient001/s0016lre
Generating record list for: patient002/s0015lre
Generating record list for: patient003/s0017lre
Generating record list for: patient004/s0020are
Generating record list for: patient004/s0020bre
Generating record list for: patient005/s0021are
Generating record list for: patient005/s0021bre
Generating record list for: patient005/s0025lre
Generating record list for: patient005/s0031lre
Generating record list for: patient005/s0101lre
Generating record list for: patient006/s0022lre
Generating record list for: patient006/s0027lre
Generating record list for: patient006/s0064lre
Generating record list for: patient007/s0026lre
Generating record list for: patient007/s0029lre
Generating record list for: patient007/s0038lre
Generating record list for: patient007/s0078lre
Generating record list for: patient008/s0028lre
Generating record list for: patient008/s

In [5]:
# preprocess_to_images.py
import os, glob
import numpy as np
import wfdb
from scipy.signal import butter, filtfilt, resample
from pyts.image import RecurrencePlot
from PIL import Image

DATA_DIR = 'data/ptbdb'        # where wfdb.dl_database put files
OUT_DIR = 'images'             # where we store images (train/val split later)
SEG_SECONDS = 5                # segment length in seconds
TARGET_FS = 250                # downsample to this sampling rate
IMG_SIZE = (224,224)

def bandpass(sig, fs, low=0.5, high=40, order=4):
    nyq = 0.5*fs
    b,a = butter(order, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, sig)

def find_dat_records(base_dir):
    dats = glob.glob(os.path.join(base_dir, '**', '*.dat'), recursive=True)
    # return list of basepaths without extension
    return [p[:-4] for p in dats]

def make_rp_image(segment, out_path):
    # segment: 1D numpy array
    rp = RecurrencePlot()  # default params; tune if needed
    X = rp.fit_transform(segment.reshape(1, -1))[0]  # 2D array
    # normalize and convert to PIL
    Xn = ((X - X.min()) / (X.max() - X.min() + 1e-12) * 255).astype('uint8')
    img = Image.fromarray(Xn)
    img = img.resize(IMG_SIZE)
    img = img.convert('RGB')
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    img.save(out_path)

def process_record(rec_basepath, label_fn, out_root='images'):
    # label_fn(record_basepath) -> 'healthy' or 'abnormal' (you must supply mapping)
    try:
        rec = wfdb.rdrecord(rec_basepath)
    except Exception as e:
        print('skip', rec_basepath, e); return

    fs = rec.fs
    sig = rec.p_signal[:, 0]  # choose lead 0; change if you prefer
    sig = bandpass(sig, fs)
    # downsample to TARGET_FS
    if fs != TARGET_FS:
        n = int(len(sig) * TARGET_FS / fs)
        sig = resample(sig, n)
        fs = TARGET_FS

    seg_len = SEG_SECONDS * fs
    nsegs = len(sig) // seg_len
    label = label_fn(rec_basepath)
    for i in range(nsegs):
        seg = sig[int(i*seg_len):int((i+1)*seg_len)]
        if len(seg) < seg_len:
            continue
        recname = os.path.basename(rec_basepath)
        out_path = os.path.join(out_root, label, f'{recname}_seg{i}.png')
        make_rp_image(seg, out_path)

# Example: you must provide a mapping from record -> label (healthy/abnormal).
# label_fn can parse clinical files or a CSV you create. For demo:

def demo_label_fn(rec_basepath):
    """
    Assigns a label (healthy / abnormal) based on the clinical summary
    inside the .hea file of each PTB record.
    """
    hea_path = rec_basepath + '.hea'
    try:
        with open(hea_path, 'r') as f:
            header = f.read().lower()  # make lowercase for easy search

        # Check if healthy control is mentioned
        if "healthy control" in header:
            return "healthy"
        else:
            return "abnormal"

    except Exception as e:
        print(f"Warning: Could not read header for {rec_basepath}: {e}")
        return "unknown"




In [7]:
DATA_DIR = 'data/ptbdb'
OUT_DIR = 'images'

recs = find_dat_records(DATA_DIR)
for r in recs:
    process_record(r, demo_label_fn, out_root=OUT_DIR)


In [10]:
# train_resnet.py (or run in Colab cell)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

IMG_SIZE = (224,224)
BATCH = 16
train_dir = 'images'  # expects subfolders 'healthy' and 'abnormal'

datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             validation_split=0.2,
                             horizontal_flip=False,  # flips may not make sense; tune
                             rotation_range=10)

train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, class_mode='binary',
                                        subset='training', batch_size=BATCH)
val_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, class_mode='binary',
                                      subset='validation', batch_size=BATCH)

base = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))
base.trainable = False   # freeze

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.4)(x)
out = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=base.input, outputs=out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

os.makedirs('models', exist_ok=True)
ckpt = ModelCheckpoint('models/best_resnet_ecg.h5', monitor='val_loss', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)

history = model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[ckpt, es, rlr])
# Save final model (weights + architecture)
model.save('models/resnet_ecg_initial.h5')


Found 9497 images belonging to 2 classes.
Found 2374 images belonging to 2 classes.
Epoch 1/5
594/594 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8171 - auc: 0.7626 - loss: 0.4404

594/594 ━━━━━━━━━━━━━━━━━━━━ 1986s 3s/step - accuracy: 0.8172 - auc: 0.7627 - loss: 0.4402 - val_accuracy: 0.8606 - val_auc: 0.8647 - val_loss: 0.3325 - learning_rate: 1.0000e-04
Epoch 2/5
594/594 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8913 - auc: 0.9244 - loss: 0.2419

594/594 ━━━━━━━━━━━━━━━━━━━━ 1992s 3s/step - accuracy: 0.8913 - auc: 0.9244 - loss: 0.2419 - val_accuracy: 0.8783 - val_auc: 0.8883 - val_loss: 0.2823 - learning_rate: 1.0000e-04
Epoch 3/5
594/594 ━━━━━━━━━━━━━━━━━━━━ 1957s 3s/step - accuracy: 0.9022 - auc: 0.9401 - loss: 0.2228 - val_accuracy: 0.8711 - val_auc: 0.8843 - val_loss: 0.3207 - learning_rate: 1.0000e-04
Epoch 4/5
594/594 ━━━━━━━━━━━━━━━━━━━━ 1967s 3s/step - accuracy: 0.9149 - auc: 0.9497 - loss: 0.2002 - val_accuracy: 0.8749 - val_auc: 0.8910 - val_loss: 0.2877 - learning_rate: 1.0000e-04
Epoch 5/5
594/594 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9233 - auc: 0.9565 - loss: 0.1883

594/594 ━━━━━━━━━━━━━━━━━━━━ 1970s 3s/step - accuracy: 0.9233 - auc: 0.9565 - loss: 0.1883 - val_accuracy: 0.8724 - val_auc: 0.8950 - val_loss: 0.2794 - learning_rate: 1.0000e-04
